In [2]:
from random import shuffle
import phrasemachine
import glob
import io
from bs4 import BeautifulSoup
import requests
from time import sleep
from collections import Counter
from twitter_dm.utility.general_utils import tab_stringify_newline as tsn
import io
import os
import yaml
import pandas as pd
import numpy as np

import seaborn as sns
%matplotlib inline

PATH_TO_ARTICLES = "/home/tcoleman/reportersOnTwitter/articles/"

phrasemachine.get_phrases('phrasemachine is working now sweet',tagger='spacy',minlen=1,maxlen=5)

In [4]:
# Set of left/right leaning terms

dem_terms = set(['assault weapons', 'bigotry', 'cancer screenings', 'carbon pollution', 'cents for every dollar', 'childhood education', 'communities of color', 'democratic caucus', 'early childhood education', 'effects of climate change', 'equal access', 'equal pay', 'equal rights', 'equal work', 
 'extreme weather', 'fair pay', 'federal investment', 'federal minimum wage', 'gun violence prevention', 'income inequality', 'inequality', 'lgbt', 'lgbtq', 'marriage equality', 'mass shootings', 'millions of women', 'pay for equal work', 'paycheck fairness', 'pell grants', 'people of color', 
'preventive health', 'public transit', 'reproductive health', 'reproductive rights', 'sexual orientation', 'universal background checks', 'violence prevention', 'voting rights act', 'wage gap', 
'access to the ballot box', 'affordable housing', 'big oil', 'billionaires', 'birth control', 'campaign finance', 
'civil rights movement', 'clean energy', 'college affordability', 'community policing', 'comprehensive immigration reform', 'confederate flag', 'consumer protections', 'crisis in flint', 'deadliest mass shooting', 'dreamers', 'early voting'])

rep_terms = set(['agreement review act', 'american security against foreign enemies', 'amnesty', 'balanced budget', 
                 'big government', 'burdensome regulations', 'bureaucracy', 'bureaucrats', 'capable unborn child protection act', 
                 'common core', 'compliance costs', 'deal with iran', 'death tax', 'death to america', 'defund planned parenthood',
                 'detainees', 'executive overreach', 'federal bureaucrats', 'foreign enemies', 'government regulations', 
                 'guantanamo bay', 'healthcare freedom reconciliation', 'human life', 'illegal immigrants', 'iran deal', 
                 'iran nuclear agreement review', 'islamic terrorism', 'job creators', 'mandates', 'medical device tax', 
                 'national debt', 'new regulations', 'nuclear agreement review act', 'nuclear deal', 'overreach',
                 'overregulation', 'planned parenthood act', 'power grab', 'power of the purse', 'red tape', 'regulatory burden',
                 'regulatory burdens', 'regulatory overreach', 'reins act', 'sanctity', 'separation of powers', 'sponsor of terror',
                 'sponsor of terrorism', 'tax code', 'tax increases', 'taxpayer funding', 'unborn child', 'unborn children',
                 'unelected bureaucrats', 'washington bureaucrats', 'waters of the united'])

all_terms = dem_terms | rep_terms

In [11]:
# function to extract phrases from an article for a given reporter 
def gen_res_reporter(dirname):
    print dirname
    files = glob.glob(dirname+"/*")
    res = []
    n_fils = 0
    for fil in files:
        text = io.open(fil).read()
        if len(text.split()) < 200:
            continue
        n_fils += 1
        phrases =  [ (k,v) for k,v in phrasemachine.get_phrases(text,tagger='spacy',minlen=1,maxlen=5)['counts'].items()
                    if k in all_terms]
        if len(phrases):
            res.append(phrases)
    return dirname.split("/")[-2:], res, n_fils


In [28]:
# Get authors that fit our inclusion criteria from larger set for analysis
authors_considered = pd.read_csv("data/authors_considered_agency.csv")
files_for_authors_considered = []
for i, row in authors_considered.iterrows():
    fil = os.path.join(PATH_TO_ARTICLES,row['agency'],row['author'])
    files_for_authors_considered.append(fil)

In [ ]:
from multiprocessing import Pool
p = Pool(15)
tot_res = p.map(gen_res_reporter, files_for_authors_considered)
p.close()
p.terminate()

In [31]:
df = []
for r in tot_res:
    df += [ r[0]+list(y)+[i,r[2]] for i,x in enumerate(r[1]) for y in x]

df = pd.DataFrame(df)
df.columns = ['agency','author','term','n_count','article','total_articles']

In [32]:
df[['author','agency']].nunique()

author    676
agency     25
dtype: int64

In [33]:
term_dict = pd.DataFrame( [(t,'dem') for t in dem_terms] + [(t,'rep') for t in rep_terms],columns=['term','leaning'])

In [34]:
df = pd.merge(df,term_dict,on="term")

In [35]:
df2 = df.groupby(['agency','author','leaning','total_articles']).n_count.sum().reset_index()

In [37]:
df2 = pd.pivot_table(df2, 'n_count',['author','agency','total_articles'],'leaning').reset_index()

In [38]:
df2 = df2.fillna(0)

In [40]:
df2['lo'] = np.log((df2.dem+1)/(df2.rep+1))

In [ ]:
a_count = df.groupby("author").article.nunique().reset_index()
authors = a_count[a_count.article > 10].author

agency_counts = df2[df2.author.isin(authors)].groupby("agency").size().reset_index()
agency_counts.columns = ['agency','n_journalists']
agencies = agency_counts[agency_counts.n_journalists >= 5].agency

In [ ]:
df2[df2.agency.isin(agencies) & df2.author.isin(authors)].groupby("agency").lo.mean().sort_values()

In [ ]:
df2[df2.agency.isin(agencies) & df2.author.isin(authors)].to_csv("lo.csv")